In [1]:
import json

def get_current_weather(location: str, unit: str):
	"""
	Get the current weather in a given location

	location (str): The city and state, e.g. Madrid, Barcelona
	unit (str): The unit. It can take two values; "celsius", "fahrenheit"
	"""
	if location == "Madrid":
		return json.dumps({"temperature": 25, "unit": unit})

	else:
		return json.dumps({"temperature": 58, "unit": unit})

In [2]:
get_current_weather(location="Madrid", unit="celsius")

'{"temperature": 25, "unit": "celsius"}'

In [3]:
import os
import re
from groq import Groq

MODEL = "llama3-groq-70b-8192-tool-use-preview"
GROQ_CLIENT = Groq(
    api_key=''
)

In [4]:
# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""

In [5]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]
agent_chat_history = []

user_msg = {
    "role": "user",
    "content": "What's the current temperature in Madrid, in Celsius?"
}

In [6]:
tool_chat_history.append(user_msg)
tool_chat_history

[{'role': 'system',
  'content': '\nYou are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. \nYou may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug \ninto functions. Pay special attention to the properties \'types\'. You should use those types as in a Python dict.\nFor each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:\n\n<tool_call>\n{"name": <function-name>,"arguments": <args-dict>}\n</tool_call>\n\nHere are the available tools:\n\n<tools> {\n    "name": "get_current_weather",\n    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; \'celsius\', \'fahrenheit\'",\n    "parameters": {\n        "properties": {\n            "location": {\n                "type": "str"\n            },\n   

In [7]:
agent_chat_history.append(user_msg)
agent_chat_history

[{'role': 'user',
  'content': "What's the current temperature in Madrid, in Celsius?"}]

In [8]:
output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<tool_call>
{"id": 0, "name": "get_current_weather", "arguments": {"location": "Madrid, Spain", "unit": "celsius"}}
</tool_call>


In [9]:
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)
    
    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output"

In [10]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'id': 0,
 'name': 'get_current_weather',
 'arguments': {'location': 'Madrid, Spain', 'unit': 'celsius'}}

In [11]:
result = get_current_weather(**parsed_output["arguments"])
result

'{"temperature": 58, "unit": "celsius"}'

In [12]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

In [13]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

'The current temperature in Madrid is 14 degrees Celsius.'